# 0. Import, data read, function definition

In [12]:
import os, sys
import numpy as no
import pandas as pd
import sklearn

In [11]:
!ls data

sample_submission.csv  test.csv  train.csv


In [29]:
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")

print(train.columns)

Index(['id', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14',
       'target'],
      dtype='object')


In [33]:
X_test = test.iloc[:,1:].values
X_train = train.iloc[:,1:-1].values
Y_train = train.iloc[:,-1].values

print(X_test.shape, X_train.shape, Y_train.shape) 
# our input space has 14 dimensions
# with 200 000 test examples, and 300 000 train examples
# output space has 1 dimension

(200000, 14) (300000, 14) (300000,)


In [ ]:
# 1. MK